# Guide: Building a Loan with _cred_

_cred_ is an open source Python package for modeling commerial real estate debt. This vignette is a step-by-step 
guide to creating a loan object and calculating the pro forma debt schedule including dates, payments, and coverage metrics. In this tutorial, we are going to build a debt model for a loan originated today based on live end of day interest rates. Click [here](https://colab.research.google.com/github/jordanhitchcock/cred_guides/blob/master/loan_schedule_quickstart.ipynb) to open and run this notebook in your browser with Google Colab.

This guide was written for readers that are familiar with CRE debt structures but are new to both Python and the _cred_ package. It provides full support to follow along including environment setup and installation. For directions on how to set up the environment, start by skipping down the section called [Setting up the Python environment](#pythonenv) at the bottom.
__Links:__ [__Github__](https://github.com/jordanhitchcock/cred), [__Docs__](https://cred.readthedocs.io/en/latest/)

## Installation and imports
One of the great advantages of _cred_ over Excel is the ability to easily connect to outside sources of information such as cash flow projections pulled from leasing software or third-party market data providers. Here we use a helper package called creatively called `treasurydotgovyc` to pull the end-of-day Treasury yield curve from treasury.gov.

In [1]:
!pip install python-dateutil > /dev/null
!pip install git+https://github.com/jordanhitchcock/treasurydotgovyc > /dev/null
!pip install git+https://github.com/jordanhitchcock/cred > /dev/null
from datetime import date
from dateutil.relativedelta import relativedelta
from treasurydotgovyc import YieldCurve
from cred import FixedRateBorrowing, Monthly

  Running command git clone -q https://github.com/jordanhitchcock/treasurydotgovyc /private/var/folders/3x/0m97cfv94pq0bw4hq39_6jv80000gn/T/pip-req-build-g55xye76
  Running command git clone -q https://github.com/jordanhitchcock/cred /private/var/folders/3x/0m97cfv94pq0bw4hq39_6jv80000gn/T/pip-req-build-nqkrg79f


<a id='loanschedule'><a/>
## Creating the loan schedule

The three lines below are all it takes to fetch the most recent yield curve, initialize a loan, and print the cash flow schedule!

In [2]:
yc = YieldCurve()

loan = FixedRateBorrowing(start_date=date.today(),
                          end_date=date.today() + relativedelta(months=12),
                          freq=Monthly(1),
                          initial_principal=100_000_000,
                          coupon=yc.yield_for_delta(relativedelta(years=1)) + 0.02)

loan.schedule()

,start_date,end_date,payment_date,bop_principal,interest_rate,interest_payment,principal_payment,payment,eop_principal
index,,,,,,,,,
0,2020-04-16,2020-05-16,2020-05-16,100000000,0.0217,180833.333333,0,1.808333e+05,100000000
1,2020-05-16,2020-06-16,2020-06-16,100000000,0.0217,186861.111111,0,1.868611e+05,100000000
2,2020-06-16,2020-07-16,2020-07-16,100000000,0.0217,180833.333333,0,1.808333e+05,100000000
3,2020-07-16,2020-08-16,2020-08-16,100000000,0.0217,186861.111111,0,1.868611e+05,100000000
4,2020-08-16,2020-09-16,2020-09-16,100000000,0.0217,186861.111111,0,1.868611e+05,100000000
5,2020-09-16,2020-10-16,2020-10-16,100000000,0.0217,180833.333333,0,1.808333e+05,100000000
6,2020-10-16,2020-11-16,2020-11-16,100000000,0.0217,186861.111111,0,1.868611e+05,100000000
7,2020-11-16,2020-12-16,2020-12-16,100000000,0.0217,180833.333333,0,1.808333e+05,100000000
8,2020-12-16,2021-01-16,2021-01-16,100000000,0.0217,186861.111111,0,1.868611e+05,100000000


Let's break that down (show/hide cell output by selecting and the section above and typing "o").
1. We initialize a new `YieldCurve` object which we will use later on to calculate the coupon rate. When this object is initialized, it sends a get request to [treasury.gov](https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/XmlView.aspx?data=yield) to get the most recent EoD yield curve and parses the XML response.

2. Next we create a new fixed rate loan and initialize it with the following values:

Parameter | Value | Details
:- | :- | :-
`start_date` | Today | `date.today()` returns today's date.
`end_date` | One year from today | This example loan has a one year term. Finds today's date and adds one year.
`freq` | 1 month | Date offset object that defines interest period frequency.
`initial_principal` | $100 million | Initial principal balance. Interest only, so the principal amount does not change.
`coupon` | Interp. T rate + 2.00% | `yc.yield_for_delta` asks the `yc` object to return the interpolated Treasury rate for the `relativedelta` offset parameter, which in this case is 1 year to match the term of the loan.

3. Finally, the `.schedule()` method returns a `pd.DataFrame` with the loan schedule.


## Creating a more complicated loan
The example above is incredibly simple. In practice, loan schedules often aren't as tidy. The _cred_ package simplifies those complexities. This next example highlights how to handle common issues either through convenience functions or custom implementation.

First, let's redefine our starting assumptions and recreate the first loan object. We will modify this object to look more like a typical lifeco loan that you might actually see in the wild.

In [3]:
closing_date = date.today()
term = Monthly(months=12)
maturity = closing_date + term
freq = relativedelta(months=1)
initial_principal = 100_000_000
coupon = yc.yield_for_delta(term) + 0.02

loan = FixedRateBorrowing(start_date=closing_date,
                         end_date=maturity,
                         freq=freq,
                         initial_principal=initial_principal,
                         coupon=coupon)

The term of our initial example conveniently equals exactly one year. Let's change the loan to a two year term beginning on the 15th of next month, rolling on the last day of the month, and maturing on the 15th day of the month in which it matures. This means we will have beginning and ending stub interest periods.

In [4]:
loan.start_date = date.today() + relativedelta(months=1, day=15) 
loan.first_reg_start = closing_date + relativedelta(months=1, day=31) # If the next month has < 31 days, will set to last day
loan.end_date = loan.start_date + Monthly(months=24)

loan.schedule()

,start_date,end_date,payment_date,bop_principal,interest_rate,interest_payment,principal_payment,payment,eop_principal
index,,,,,,,,,
0,2020-05-15,2020-05-31,2020-05-15,100000000,0.0217,96444.444444,0,9.644444e+04,100000000
1,2020-05-31,2020-06-30,2020-06-30,100000000,0.0217,180833.333333,0,1.808333e+05,100000000
2,2020-06-30,2020-07-31,2020-07-31,100000000,0.0217,186861.111111,0,1.868611e+05,100000000
3,2020-07-31,2020-08-31,2020-08-31,100000000,0.0217,186861.111111,0,1.868611e+05,100000000
4,2020-08-31,2020-09-30,2020-09-30,100000000,0.0217,180833.333333,0,1.808333e+05,100000000
5,2020-09-30,2020-10-31,2020-10-31,100000000,0.0217,186861.111111,0,1.868611e+05,100000000
6,2020-10-31,2020-11-30,2020-11-30,100000000,0.0217,180833.333333,0,1.808333e+05,100000000
7,2020-11-30,2020-12-31,2020-12-31,100000000,0.0217,186861.111111,0,1.868611e+05,100000000
8,2020-12-31,2021-01-31,2021-01-31,100000000,0.0217,186861.111111,0,1.868611e+05,100000000


<div class="alert alert-info">

**Note on initialization and stored values:** We are directly modifying properties on Borrowing object, but for each property that we are change there is parameter (usually) with the same name that you can define at initialization.
    
Period values are not stored, so you can safely change any Borrowing property and the schedule will rebuild as expected. For performance, period values are cached during execution of the _.schedule()_ method. You can manually tell it to cache values by using the _with_ context manager, but be careful about changing object properties.

</div>

Great, now we have our dates all line up with the expected closing timeline. Notice that by default the payment date for the initial stub period is on the closing date which is common in practice. However, payment dates are unadjusted and might currently fall on weekends or holidays. Let's fix this by defining a modified following convention with New York business days.

In [5]:
from cred import modified_following, FederalReserveHolidays

loan.adjust_pmt_date = modified_following
loan.holidays = FederalReserveHolidays()

Note that we adjusted the payment dates but not the interest period calculation dates. Depending on the specific definitions in the loan documents, the interest period starting and ending dates may or may not also follow an adjustment convention. This largely depends on the lending market, lender, and counsel drafting the loan docs.

The default interest calculation uses an Actual / 360 day count convention. Since this loan is from a lifeco and priced over the Treasury rate instead of LIBOR or the swap rate, we will change that to a 30 / 360 day count convention. The 30 / 360 calculation method used in _cred_ is the method Excel uses.

In [6]:
from cred import thirty360

loan.year_frac = thirty360

Finally, since insurance companies are risk averse lenders, we'll add amortization on a 30-year schedule for second year to meet their concerns of takeout risk.

In [7]:
loan.amort_periods = 360
loan.io_periods = 12
loan.schedule()

,start_date,end_date,payment_date,bop_principal,interest_rate,interest_payment,principal_payment,payment,eop_principal
index,,,,,,,,,
0,2020-05-15,2020-05-31,2020-05-15,1.000000e+08,0.0217,96444.444444,0.000000e+00,9.644444e+04,1.000000e+08
1,2020-05-31,2020-06-30,2020-06-30,1.000000e+08,0.0217,180833.333333,1.973451e+05,3.781785e+05,9.980265e+07
2,2020-06-30,2020-07-31,2020-07-31,9.980265e+07,0.0217,180476.467565,1.977020e+05,3.781785e+05,9.960495e+07
3,2020-07-31,2020-08-31,2020-08-31,9.960495e+07,0.0217,180118.956465,1.980595e+05,3.781785e+05,9.940689e+07
4,2020-08-31,2020-09-30,2020-09-30,9.940689e+07,0.0217,179760.798865,1.984177e+05,3.781785e+05,9.920848e+07
5,2020-09-30,2020-10-31,2020-10-30,9.920848e+07,0.0217,179401.993597,1.987765e+05,3.781785e+05,9.900970e+07
6,2020-10-31,2020-11-30,2020-11-30,9.900970e+07,0.0217,179042.539490,1.991359e+05,3.781785e+05,9.881056e+07
7,2020-11-30,2020-12-31,2020-12-31,9.881056e+07,0.0217,178682.435369,1.994960e+05,3.781785e+05,9.861107e+07
8,2020-12-31,2021-01-31,2021-01-29,9.861107e+07,0.0217,178321.680061,1.998568e+05,3.781785e+05,9.841121e+07


## Putting it all together

Congratulations, you've created a customized loan schedule using _cred_ !  You can export the loan schedule to an Excel file in your working directory by running `loan.schedule().to_excel('my_file_name.xlsx')`. The last code block below compiles everything we've done into one initialization statement and formats the output to look nicer.

Visit the documentation at [cred.readthedocs.io](https://cred.readthedocs.io/en/latest/) to learn more about how you can extend loan objects to build dynamic floating rate schedules, calculate covenant tests, and evaluate other loan metrics using subclasses.

In [8]:
from cred import FixedRateBorrowing, Monthly, thirty360, FederalReserveHolidays, modified_following

loan = FixedRateBorrowing(start_date=date.today() + relativedelta(months=1, day=15),
                          first_reg_start=date.today() + relativedelta(months=1, day=31),
                          end_date=date.today() + relativedelta(months=25, day=15),
                          freq=Monthly(),
                          initial_principal=100_000_000,
                          coupon=yc.yield_for_delta(relativedelta(months=12)) + 0.02,
                          amort_periods=360,
                          pmt_convention=modified_following,
                          year_frac=thirty360,
                          holidays=FederalReserveHolidays())

# formatting
schedule = loan.schedule()
schedule.columns = ['Start Date', 'End Date', 'Payment Date', 'Start Principal', 'Interest Rate', 'Interest Payment', 'Principal Payment', 'Total Payment', 'End Principal']
schedule = schedule.style.format({'Start Principal': '${:,.0f}', 'Interest Rate': '{:.2%}', 'Interest Payment': '${:,.0f}', 'Principal Payment': '${:,.0f}', 'Total Payment': '${:,.0f}', 'End Principal': '${:,.0f}'})

schedule

,Start Date,End Date,Payment Date,Start Principal,Interest Rate,Interest Payment,Principal Payment,Total Payment,End Principal
index,,,,,,,,,
0,2020-05-15,2020-05-31,2020-05-15,"$100,000,000",2.17%,"$96,444",$0,"$96,444","$100,000,000"
1,2020-05-31,2020-06-30,2020-06-30,"$100,000,000",2.17%,"$180,833","$197,345","$378,178","$99,802,655"
2,2020-06-30,2020-07-31,2020-07-31,"$99,802,655",2.17%,"$180,476","$197,702","$378,178","$99,604,953"
3,2020-07-31,2020-08-31,2020-08-31,"$99,604,953",2.17%,"$180,119","$198,060","$378,178","$99,406,893"
4,2020-08-31,2020-09-30,2020-09-30,"$99,406,893",2.17%,"$179,761","$198,418","$378,178","$99,208,476"
5,2020-09-30,2020-10-31,2020-10-30,"$99,208,476",2.17%,"$179,402","$198,776","$378,178","$99,009,699"
6,2020-10-31,2020-11-30,2020-11-30,"$99,009,699",2.17%,"$179,043","$199,136","$378,178","$98,810,563"
7,2020-11-30,2020-12-31,2020-12-31,"$98,810,563",2.17%,"$178,682","$199,496","$378,178","$98,611,067"
8,2020-12-31,2021-01-31,2021-01-29,"$98,611,067",2.17%,"$178,322","$199,857","$378,178","$98,411,211"


***

<a id='pythonenv'><a/>
## Setting up the Python environment

This assumes Python 3 is already installed. If not, install from [python.org](https://www.python.org). To check if it's already installed, run `which python` in the terminal.

Open up Terminal (mac) or Command Prompt (Windows) and navigate to the location where you want to set up your project. Type:

``` 
mkdir cred_guide
cd cred_guide 
```

to create a new folder called "cred_guide" and then move inside the new folder. This is where we will create our virtual environment and store our project. 

Next, we'll set up our virtual environment. Virtual environments are isolated environments where you can install and run libraries for a specific project without worrying about messing up packages or dependencies for other projects. It isn't required, but it's best practice. Real Python has a great [primer on virtual environments](https://realpython.com/python-virtual-environments-a-primer/). We'll walk through Virtualenv here since it comes with the base version of Python 3, but there are several ways to set up virtual environments. [Conda](https://docs.conda.io/en/latest/) is another popular option.

Create and activatethe virtual environment by running the following in the termnial.

```
python -m venv venv

// activate on mac
source venv/bin/activate

// activate on windows
C:\path\to\cred_guide\venv\Scripts\activate.bat
```

The terminal prompt should change to something like `(base) :UserName$` to show that the virtual environment has been activated.

Finally, install the python packages necessary to run this tutorial:

```
pip install python-dateutil
pip install git+https://github.com/jordanhitchcock/treasurydotgovyc
pip install git+https://github.com/jordanhitchcock/cred

python  # Starts Python. From here on out, everything is Python code.

from datetime import date
from dateutil.relativedelta import relativedelta
 
from treasurydotgovyc import YieldCurve
from cred import FixedRateBorrowing, Monthly
```

Great, you're ready to jump back in at the [Creating the loan schedule](#loanschedule) section!